In [2]:
#import necessary packages
import requests
import numpy as np
import pandas as pd
from selectorlib import Extractor
from dateutil import parser as dateparser
import pg8000
from config import config
import schedule

In [3]:
#get url for products
url = {"FR_reviews": "https://www.amazon.com/Fire-TV-Stick-Previous-Generation/product-reviews/B00ZV9RDKK/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=%s",
       "HP_reviews": "https://www.amazon.com/Harry-Potter-Sorcerers-Stone-Rowling/product-reviews/059035342X/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&&sortBy=recentpageNumber=%s",
       "CAH_reviews": "https://www.amazon.com/Cards-Against-Humanity-LLC-CAHUS/product-reviews/B004S8F7QM/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&&sortBy=recentpageNumber=%s",
       "SP_reviews": "https://www.amazon.com/amFilm-Screen-Protector-iPhone-Tempered/product-reviews/B01415QHYW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&&sortBy=recentpageNumber=%s",
       "BS_reviews": "https://www.amazon.com/Mellanni-Bed-Sheet-Set-Hypoallergenic/product-reviews/B00NLLUMOE/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&&sortBy=recentpageNumber=%s"}

#only take the first 200 pages to gather enough data
num_pages = 200

#using selectorlib to parse url html data
extractor = Extractor.from_yaml_file('selectors.yml')

In [4]:
def scrape(url):
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    # Download the page using requests
    #print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    data = extractor.extract(r.text,base_url=url)
    reviews = []
    columns_to_remove = ["images", "found_helpful", "variant", "verified_purchase"]
    for r in data['reviews']:
        r["product"] = data["product_title"]
        r['url'] = url
        r['rating'] = pd.to_numeric(r['rating'].split(' out of')[0], errors='coerce', downcast='float')
        date_posted = r['date'].split('on ')[-1]
        r['date'] = pd.to_datetime(dateparser.parse(date_posted).strftime('%d %b %Y'))
        [r.pop(key) for key in columns_to_remove]
        reviews.append(r)
    return reviews

In [5]:
def create_reviews():
    """ Connect to the PostgreSQL database server """

    # create sql statement
    create_review_tables = (
            """
            CREATE TABLE IF NOT EXISTS FR_reviews(
                reviews_id SERIAL PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT NOT NULL,
                date DATE NOT NULL,
                author VARCHAR(255),
                rating FLOAT NOT NULL,
                product VARCHAR(255) NOT NULL,
                url  VARCHAR(255) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS HP_reviews(
                reviews_id SERIAL PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT NOT NULL,
                date DATE NOT NULL,
                author VARCHAR(255),
                rating FLOAT NOT NULL,
                product VARCHAR(255) NOT NULL,
                url  VARCHAR(255) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS CAH_reviews(
                reviews_id SERIAL PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT NOT NULL,
                date DATE NOT NULL,
                author VARCHAR(255),
                rating FLOAT NOT NULL,
                product VARCHAR(255) NOT NULL,
                url  VARCHAR(255) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS SP_reviews(
                reviews_id SERIAL PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT NOT NULL,
                date DATE NOT NULL,
                author VARCHAR(255),
                rating FLOAT NOT NULL,
                product VARCHAR(255) NOT NULL,
                url  VARCHAR(255) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS BS_reviews(
                reviews_id SERIAL PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT NOT NULL,
                date DATE NOT NULL,
                author VARCHAR(255),
                rating FLOAT NOT NULL,
                product VARCHAR(255) NOT NULL,
                url  VARCHAR(255) NOT NULL
            )
            """
    )

    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()   

        # execute sql statements
        for command in create_review_tables:
            cur.execute(command)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [6]:
def insert_FR_reviews(data):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	    # create sql statement
        insert_reviews = (
            """
            INSERT INTO FR_reviews(title, content, date, author, rating, product, url) VALUES(%s, %s, %s, %s, %s, %s, %s)
            """
        )

        # execute sql statements
        for rev in data.values.tolist():
            cur.execute(insert_reviews, rev)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

def insert_HP_reviews(data):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	    # create sql statement
        insert_reviews = (
            """
            INSERT INTO HP_reviews(title, content, date, author, rating, product, url) VALUES(%s, %s, %s, %s, %s, %s, %s)
            """
        )

        # execute sql statements
        for rev in data.values.tolist():
            cur.execute(insert_reviews, rev)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

def insert_CAH_reviews(data):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	    # create sql statement
        insert_reviews = (
            """
            INSERT INTO CAH_reviews(title, content, date, author, rating, product, url) VALUES(%s, %s, %s, %s, %s, %s, %s)
            """
        )

        # execute sql statements
        for rev in data.values.tolist():
            cur.execute(insert_reviews, rev)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
        
def insert_SP_reviews(data):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	    # create sql statement
        insert_reviews = (
            """
            INSERT INTO SP_reviews(title, content, date, author, rating, product, url) VALUES(%s, %s, %s, %s, %s, %s, %s)
            """
        )

        # execute sql statements
        for rev in data.values.tolist():
            cur.execute(insert_reviews, rev)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

def insert_BS_reviews(data):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg8000.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	    # create sql statement
        insert_reviews = (
            """
            INSERT INTO BS_reviews(title, content, date, author, rating, product, url) VALUES(%s, %s, %s, %s, %s, %s, %s)
            """
        )

        # execute sql statements
        for rev in data.values.tolist():
            cur.execute(insert_reviews, rev)
        
        conn.commit()
        
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [7]:
def update_FR_reviews():
    """ update vendor name based on the vendor id """
    all_reviews = pd.DataFrame(get_reviews(url['FR_reviews'], num_pages))
    sql = """ 
        SELECT date FROM FR_reviews
    """
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = pg8000.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # get current table state statement
        cur.execute(sql)
        
        dates = []
        row = cur.fetchone()
        while row is not None:
            dates= dates + row
            row = cur.fetchone()
        dates = pd.to_datetime(pd.Series(dates))

        updated_dates = set(all_reviews['date'].tolist())
        current_dates = set(dates.tolist())
        if updated_dates != current_dates:
            insert_FR_reviews(all_reviews.loc[~all_reviews['date'].isin(list(updated_dates.intersection(current_dates)))])

        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_HP_reviews():
    """ update vendor name based on the vendor id """
    all_reviews = pd.DataFrame(get_reviews(url['HP_reviews'], num_pages))
    sql = """ 
        SELECT date FROM HP_reviews
    """
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = pg8000.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # get current table state statement
        cur.execute(sql)
        
        dates = []
        row = cur.fetchone()
        while row is not None:
            dates= dates + row
            row = cur.fetchone()
        dates = pd.to_datetime(pd.Series(dates))

        updated_dates = set(all_reviews['date'].tolist())
        current_dates = set(dates.tolist())
        if updated_dates != current_dates:
            insert_HP_reviews(all_reviews.loc[~all_reviews['date'].isin(list(updated_dates.intersection(current_dates)))])

        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_CAH_reviews():
    """ update vendor name based on the vendor id """
    all_reviews = pd.DataFrame(get_reviews(url['CAH_reviews'], num_pages))
    sql = """ 
        SELECT date FROM CAH_reviews
    """
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = pg8000.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # get current table state statement
        cur.execute(sql)
        
        dates = []
        row = cur.fetchone()
        while row is not None:
            dates= dates + row
            row = cur.fetchone()
        dates = pd.to_datetime(pd.Series(dates))

        updated_dates = set(all_reviews['date'].tolist())
        current_dates = set(dates.tolist())
        if updated_dates != current_dates:
            insert_CAH_reviews(all_reviews.loc[~all_reviews['date'].isin(list(updated_dates.intersection(current_dates)))])

        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_SP_reviews():
    """ update vendor name based on the vendor id """
    all_reviews = pd.DataFrame(get_reviews(url['SP_reviews'], num_pages))
    sql = """ 
        SELECT date FROM SP_reviews
    """
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = pg8000.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # get current table state statement
        cur.execute(sql)
        
        dates = []
        row = cur.fetchone()
        while row is not None:
            dates= dates + row
            row = cur.fetchone()
        dates = pd.to_datetime(pd.Series(dates))

        updated_dates = set(all_reviews['date'].tolist())
        current_dates = set(dates.tolist())
        if updated_dates != current_dates:
            insert_SP_reviews(all_reviews.loc[~all_reviews['date'].isin(list(updated_dates.intersection(current_dates)))])

        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_BS_reviews():
    """ update vendor name based on the vendor id """
    all_reviews = pd.DataFrame(get_reviews(url['BS_reviews'], num_pages))
    sql = """ 
        SELECT date FROM BS_reviews
    """
    conn = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = pg8000.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # get current table state statement
        cur.execute(sql)
        
        dates = []
        row = cur.fetchone()
        while row is not None:
            dates= dates + row
            row = cur.fetchone()
        dates = pd.to_datetime(pd.Series(dates))

        updated_dates = set(all_reviews['date'].tolist())
        current_dates = set(dates.tolist())
        if updated_dates != current_dates:
            insert_BS_reviews(all_reviews.loc[~all_reviews['date'].isin(list(updated_dates.intersection(current_dates)))])

        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, pg8000.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [8]:
def get_reviews(url, num_pages):

    url_pages = []
    for x in np.arange(1, num_pages):
        page = str(x)
        url_pages.append(url % page)

    reviews = []
    for x in range(num_pages-1):
        reviews= reviews + (scrape(url_pages[x]))
    return reviews

In [10]:
FR_reviews = pd.DataFrame((get_reviews(url['FR_reviews'], num_pages)))
HP_reviews = pd.DataFrame((get_reviews(url['HP_reviews'], num_pages)))
CAH_reviews = pd.DataFrame((get_reviews(url['CAH_reviews'], num_pages)))
SP_reviews = pd.DataFrame((get_reviews(url['SP_reviews'], num_pages)))
BS_reviews = pd.DataFrame((get_reviews(url['BS_reviews'], num_pages)))

KeyboardInterrupt: 

In [49]:
create_reviews()

Connecting to the PostgreSQL database...
Database connection closed.


In [50]:
insert_FR_reviews(FR_reviews)
insert_HP_reviews(HP_reviews)
insert_CAH_reviews(CAH_reviews)
insert_SP_reviews(SP_reviews)
insert_BS_reviews(BS_reviews)

Connecting to the PostgreSQL database...
Database connection closed.


In [ ]:
import schedule
import time
import web_scrape

updater = web_scrape.WebScrape()
schedule.every.monday.at("23:45").do(updater.update_FR_reviews)
schedule.every.monday.at("23:45").do(updater.update_HP_reviews)
schedule.every.monday.at("23:45").do(updater.update_CAH_reviews)
schedule.every.monday.at("23:45").do(updater.update_SP_reviews)
schedule.every.monday.at("23:45").do(updater.update_BS_reviews)

while True:
    schedule.run_pending
    time.sleep(1)